In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()
import sys
openai.api_key = os.getenv("OAI_KEY")
brave_key = os.getenv("BRAVE_KEY")
os.environ["OPENAI_API_KEY"]= os.getenv("OAI_KEY")
client = OpenAI()  


### Create Retriever

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import ArxivLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

docs = ArxivLoader(query ="text query here", load_max_docs=2).load()
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=350, chunk_overlap=50
)

chunked_docs = text_splitter.split_documents(docs)

vector_store = FAISS.from_documents(
    documents=chunked_docs,
    embedding=OpenAIEmbeddings(model="text-embedding-3-small"),
)

retriever = vector_store.as_retriever()

#### Create Prompt Template

In [ ]:
from langchain_core.prompts import ChatPromptTemplate


RAG_PROMPT= """\
    Use the following context to answer the user's query. If you cannot answer the question, please respond with 'I do not know'
    
    Question:
    {question}
    
    Context:
    {context}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)


#### Setup Generation Model (GPT-3.5)

In [ ]:
from langchain_openai import ChatOpenAI

openai_chat_model = ChatOpenAI(model="gpt-3.5-turbo") 

In [ ]:
# TODO: EXPLORE LCEL CHAINS

from operator import itemgetter
from langchain.schema.output_parser import SrcOutputParser
from langchain.schema.runnable import RunnablePassthrough

rag_chain = (
    {'context': itemgetter('question') | retriever, 'question': itemgetter('question')}
    | RunnablePassthrough.assign(context=itemgetter('context'))
    | {'response': rag_prompt | openai_chat_model, 'context': itemgetter('context')}
)

In [ ]:
await rag_chain.ainvoke({"question": "What is RAG?"})

#### Add tools for LangGraph implementation

In [ ]:
from langchain_community.tools.ddg_search import DuckDuckGoSearch
from langchain_community.tools.arxiv.tool import ArxivQueryRun

tools_list = [
    DuckDuckGoSearch(),
    ArxivQueryRun()
]

In [ ]:
from langgraph.prebuilt import ToolExecutor
tool_executor = ToolExecutor(tools_list)


In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.utils.function_calling import convert_to_openai_function

# Get deterministic outputs from gpt
model = ChatOpenAI(temperature=0) 
functions = [convert_to_openai_function(tool) for tool in tools_list]
model = model.bind_functions(functions)

#### Create Agent State Class

In [ ]:
from typing import TypedDict, Annotated, Sequence
import operator
from langchain_core.messages import BaseMessage

class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add] #operator for documentation

#### Create graph node functions

In [ ]:
from langgraph.prebuilt import ToolInvocation
import json
from langchain_core.messages import FunctionMessage

def call_model(state):
    messages = state['messages']
    response = model.invoke(messages)
    return {"messages": [response]}

def call_tool(state):
    last_message = state['messages'][-1]
    
    action = ToolInvocation (
        tool = last_message.additional_kwargs['function_call']['name'],
        tool_input = json.loads(
            last_message.additional_kwargs['function_call']['arguments']
        )
    )
    
    response = tool_executor.invoke(action)
    
    function_msg = FunctionMessage(content=str(response), name=action.tool)
    
    return {"messages": [function_msg]}      

In [ ]:
from langgraph.graph import StateGraph, END

workflow = StateGraph(AgentState)

workflow.add_node("agent", call_model)
workflow.add_node("action", call_tool)

In [ ]:
workflow.set_entry_point("agent")

In [ ]:
# Add condition check

def should_continue(state):
    last_msg = state['messages'][-1]
    
    if 'function_call' not in last_msg.additional_kwargs:
        return 'end'
    else:
        return 'continue'

workflow.add_conditional_edges(
    'agent',
    should_continue,
    {
        'continue' : 'action',
        'end' : END
    }
)

In [ ]:
workflow.add_edge('action', 'agent')

In [ ]:
app = workflow.compile()

##### Add RAG chain to graph

In [ ]:
def convert_state_to_query(state_object):
    return {'question' : state_object['messages'][-1].content}

def convert_response_to_state(response):
    return {'messages' : [response['response']]}

langgraph_node_rag_chain = convert_state_to_query | rag_chain | convert_response_to_state

In [ ]:
from langchain_core.messages import HumanMessage

inputs = {"messages" : [HumanMessage(content="What is RAG in the context of Large Language Models? When did it break onto the scene?")]}

await langgraph_node_rag_chain.ainvoke(inputs)

In [ ]:
# Finally create RAG agent 
rag_agent = StateGraph(AgentState)

rag_agent.add_node('agent', call_model)
rag_agent.add_node('action', call_tool)
rag_agent.add_node('first_action', langgraph_node_rag_chain)

rag_agent.set_entry_point('first_action')

In [ ]:
# Add check for full answer

from langchain.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain.output_parsers.openai_tools import PydanticToolsParser
from langchain_core.utils.function_calling import convert_to_openai_tool


def is_fully_answered(state):
    
    question = state['messages'][0].content
    answer = state['messages'][-1].content

    class answered(BaseModel):
        binary_score: str = Field(description="Fully answered: 'yes' or 'no'")
        
    model = ChatOpenAI(model='gpt-4-turbo-preview', temperature=0)
    
    answered_tool = convert_to_openai_tool(answered)
    
    model = model.bind(
        tools=[answered_tool],
        tool_choice = {'type': 'function', 'function': {'name' : 'answered'}}
    )
    parser_tool = PydanticToolsParser(tools=[answered])

    prompt = PromptTemplate(
        template="""You will determine if the question is fully answered by the response.\n
        Question:
        {question}
        
        Respose:
        {answer}
        
        You will respond with either 'yes' or 'no'.""",
        input_variables=['question','answer']
    )

    complete_answer_chain = prompt | model | parser_tool
    response = complete_answer_chain.invoke({'question': question, 'answer': answer})

    if response[0].binary_score == 'no':
        return 'continue'

    return'end'

In [ ]:
rag_agent.add_conditional_edges(
    'first_action',
    is_fully_answered,
    {
        'continue' : 'agent',
        'end' : END
    }
)

In [ ]:
rag_agent.add_edge('action','agent')
rag_agent_app = rag_agent.compile()